# Deploying NVIDIA Triton Inference Server in AI Platform Prediction Custom Container (Google Cloud SDK)

In this notebook, we will walk through the process of deploying NVIDIA's Triton Inference Server into AI Platform Prediction Custom Container service in the Direct Model Server mode:

![](img/caip_triton_container_diagram_direct.jpg)


In [ ]:
PROJECT_ID='[Enter project name - REQUIRED]'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

In [ ]:
import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)

In [1]:
PROJECT_ID='tsaikevin-1236'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)

In [3]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


### Create the Artifact Registry
This will be used to store the container image for the model server Triton.

In [4]:
!gcloud beta artifacts repositories create $REPOSITORY --repository-format=docker --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [5]:
!gcloud beta auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Prepare the container
We will make a copy of the Triton container image into the Artifact Registry, where AI Platform Custom Container Prediction will only pull from during Model Version setup. The following steps will download the NVIDIA Triton Inference Server container to your VM, then upload it to your repo.

In [6]:
!docker pull nvcr.io/nvidia/$TRITON_IMAGE && \
 docker tag nvcr.io/nvidia/$TRITON_IMAGE $CAIP_IMAGE && \
 docker push $CAIP_IMAGE

20.06-py3: Pulling from nvidia/tritonserver
Digest: sha256:36f94c39221c4e19921d44296690991057bbebbb15f59dacd88e25ff331bd307
Status: Image is up to date for nvcr.io/nvidia/tritonserver:20.06-py3
nvcr.io/nvidia/tritonserver:20.06-py3
The push refers to repository [us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver]

7aefd4ea: Preparing 
ab22f50a: Preparing 
4bb8a14c: Preparing 
c357696a: Preparing 
35b111ce: Preparing 
422b8a56: Preparing 
5c73ed66: Preparing 
91761c8c: Preparing 
dcbd0b8f: Preparing 
3fad0b37: Preparing 
bca7086a: Preparing 
a1fe0dac: Preparing 
16262158: Preparing 
faf9c798: Preparing 
4dd7a77b: Preparing 
4f618f62: Preparing 
114ab5c3: Preparing 
b7588393: Preparing 
7a4b3a0b: Preparing 
3708beeb: Preparing 
c2e3c7b1: Preparing 
43d8d50a: Preparing 
9bd9798f: Preparing 
27c9414b: Preparing 
4c1700eb: Preparing 
46c23e3a: Preparing 
b877a610: Preparing 
e28a7437: Preparing 
d0584a68: Preparing 
cd722629: Preparing 
7d6d8dba: Preparing 
f0c094ce: Preparin

### Prepare model Artifacts
Clone the NVIDIA Triton Inference Server repo.

In [9]:
!git clone https://github.com/NVIDIA/triton-inference-server.git

Cloning into 'triton-inference-server'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (285/285), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 25484 (delta 149), reused 161 (delta 89), pack-reused 25199
Receiving objects: 100% (25484/25484), 14.34 MiB | 23.26 MiB/s, done.
Resolving deltas: 100% (18803/18803), done.


Create the GCS bucket where the model artifacts will be copied to.

In [10]:
!gsutil mb $MODEL_BUCKET

Creating gs://tsaikevin-1236-90830/...


Stage model artifacts and copy to bucket.

In [11]:
!mkdir model_repository

In [12]:
!cp -R triton-inference-server/docs/examples/model_repository/* model_repository/

In [ ]:
# !cd triton-inference-server && git checkout r$TRITON_VERSION

In [13]:
%cd triton-inference-server
!git checkout r$TRITON_VERSION
%cd ..
%ls -l

/home/jupyter/caip-triton/v2/simple_setup/triton-inference-server
Branch r20.06 set up to track remote branch r20.06 from origin.
Switched to a new branch 'r20.06'
/home/jupyter/caip-triton/v2/simple_setup
total 116
-rw-r--r--  1 jupyter jupyter 13652 Oct 19 08:48 get_request_body_simple.py
drwxr-xr-x  2 jupyter jupyter  4096 Oct 19 08:48 img/
drwxr-xr-x  6 jupyter jupyter  4096 Oct 26 08:10 model_repository/
-rw-r--r--  1 jupyter jupyter  1605 Oct 22 18:27 README.md
drwxr-xr-x 10 jupyter jupyter  4096 Oct 26 08:11 triton-inference-server/
-rw-r--r--  1 jupyter jupyter 60610 Oct 26 08:07 triton-simple-setup-rest.ipynb
-rw-r--r--  1 jupyter jupyter 21323 Oct 26 08:09 triton-simple-setup-sdk.ipynb


In [14]:
!./triton-inference-server/docs/examples/fetch_models.sh

+ mkdir -p model_repository/resnet50_netdef/1
+ wget -O model_repository/resnet50_netdef/1/model.netdef http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/predict_net.pb
--2020-10-26 08:11:05--  http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/predict_net.pb
Resolving download.caffe2.ai.s3.amazonaws.com (download.caffe2.ai.s3.amazonaws.com)... 52.216.233.11
Connecting to download.caffe2.ai.s3.amazonaws.com (download.caffe2.ai.s3.amazonaws.com)|52.216.233.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31649 (31K) [binary/octet-stream]
Saving to: ‘model_repository/resnet50_netdef/1/model.netdef’

model_repository/re 100%[===================>]  30.91K  --.-KB/s    in 0.03s   

2020-10-26 08:11:05 (1.03 MB/s) - ‘model_repository/resnet50_netdef/1/model.netdef’ saved [31649/31649]

+ wget -O model_repository/resnet50_netdef/1/init_model.netdef http://download.caffe2.ai.s3.amazonaws.com/models/resnet50/init_net.pb
--2020-10-26 08:11:05--  http:/

In [15]:
!gsutil -m cp -R model_repository/ $MODEL_BUCKET

Copying file://model_repository/resnet50_netdef/1/model.netdef [Content-Type=application/octet-stream]...
Copying file://model_repository/simple/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/simple/1/model.graphdef [Content-Type=application/octet-stream]...
Copying file://model_repository/resnet50_netdef/1/init_model.netdef [Content-Type=application/octet-stream]...
Copying file://model_repository/inception_graphdef/inception_labels.txt [Content-Type=text/plain]...
Copying file://model_repository/densenet_onnx/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/densenet_onnx/densenet_labels.txt [Content-Type=text/plain]...
Copying file://model_repository/inception_graphdef/config.pbtxt [Content-Type=application/octet-stream]...
Copying file://model_repository/densenet_onnx/1/model.onnx [Content-Type=application/octet-stream]...
Copying file://model_repository/inception_graphdef/1/model.graphdef [Content-Type=

In [16]:
!gsutil ls $MODEL_BUCKET/model_repository

gs://tsaikevin-1236-90830/model_repository/densenet_onnx/
gs://tsaikevin-1236-90830/model_repository/inception_graphdef/
gs://tsaikevin-1236-90830/model_repository/resnet50_netdef/
gs://tsaikevin-1236-90830/model_repository/simple/
gs://tsaikevin-1236-90830/model_repository/simple_string/


### Prepare request payload

To prepare the payload format, we have included a utility get_request_body_simple.py.  To use this utility, install the following library:

In [17]:
!pip3 install geventhttpclient

#### Prepare non-binary request payload

The first model will illustrate a non-binary payload.  The following command will create a KF Serving v2 format non-binary payload to be used with the "simple" model:

In [18]:
!python3 get_request_body_simple.py -m simple

#### Prepare binary request payload

Triton's implementation of KF Serving v2 protocol for binary data appends the binary data after the json body.  Triton requires an additional header for offset:

`Inference-Header-Content-Length: [offset]`

We have provided a script that will automatically resize the image to the proper size for ResNet-50 [224, 224, 3] and calculate the proper offset.  The following command takes an image file and outputs the necessary data structure to be use with the "resnet50_netdef" model.  Please note down this offset as it will be used later.

In [19]:
!python3 get_request_body_simple.py -m image -f triton-inference-server/qa/images/mug.jpg

(3, 224, 224)
Add Header: Inference-Header-Content-Length: 138


## Create and deploy Model and Model Version

In this section, we will deploy two models:
1. Simple model with non-binary data.  KF Serving v2 protocol specifies a json format with non-binary data in the json body itself.
2. Binary data model with ResNet-50.  Triton's implementation of binary data for KF Server v2 protocol.


### Simple model (non-binary data)

#### Create Model

AI Platform Prediction uses a Model/Model Version Hierarchy, where the Model is a logical grouping of Model Versions.  We will first create the Model.

Because the MODEL_NAME variable will be used later to specify the predict route, and Triton will use that route to run prediction on a specific model, we must set the value of this variable to a valid name of a model.  For this section, will use the "simple" model.

In [30]:
MODEL_NAME='simple'

In [31]:
!gcloud ai-platform models create $MODEL_NAME --region $REGION --enable-logging

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ml engine model [projects/tsaikevin-1236/models/simple].


In [32]:
!gcloud ai-platform models list --region $REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME    DEFAULT_VERSION_NAME
simple


#### Create Model Version

After the Model is created, we can now create a Model Version under this Model.  Each Model Version will need a name that is unique within the Model.  In AI Platform Prediction Custom Container, a {Project}/{Model}/{ModelVersion} uniquely identifies the specific container and model artifact used for inference.

In [33]:
VERSION_NAME='v01'

The following config file will be used in the Model Version creation command.

#### Command with YAML config file

In [34]:
import yaml

config_simple={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_simple.yaml', 'w') as file:
    config = yaml.dump(config_simple, file)

In [35]:
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_simple.yaml \
  --region=$REGION \
  --async

Using endpoint [https://us-central1-ml.googleapis.com/]


In [53]:
!gcloud beta ai-platform versions create 

ERROR: (gcloud.beta.ai-platform.versions.create) argument VERSION --model: Must be specified.
Usage: gcloud beta ai-platform versions create VERSION --model=MODEL [optional flags]
  optional flags may be  --accelerator | --args | --async | --command |
                         --config | --description | --env-vars |
                         --explanation-method | --framework | --health-route |
                         --help | --image | --labels | --machine-type |
                         --num-integral-steps | --num-paths | --origin |
                         --package-uris | --ports | --predict-route |
                         --prediction-class | --python-version | --region |
                         --runtime-version | --service-account |
                         --staging-bucket

For detailed information on this command and its flags, run:
  gcloud beta ai-platform versions create --help


#### To see details of the Model Version just created

In [54]:
!gcloud ai-platform versions describe $VERSION_NAME --model=$MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
acceleratorConfig:
  count: '1'
  type: NVIDIA_TESLA_T4
autoScaling:
  minNodes: 1
container:
  args:
  - tritonserver
  - --model-repository=$(AIP_STORAGE_URI)
  image: us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver:20.06-py3
  ports:
  - containerPort: 8000
createTime: '2020-10-26T08:13:29Z'
deploymentUri: gs://tsaikevin-1236-90830/model_repository
errorMessage: |
  Model server terminated: model server container terminated: exit_code: 1
  reason: "Error"
  started_at {
    seconds: 1603701437
  }
  finished_at {
    seconds: 1603701446
  }
etag: hPxbO3hKDhY=
machineType: n1-standard-4
name: projects/tsaikevin-1236/models/simple/versions/v01
routes:
  health: /v2/models/simple
  predict: /v2/models/simple/infer
state: FAILED


#### To list all Model Versions and their states in this Model

In [37]:
!gcloud ai-platform versions list --model=$MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME  DEPLOYMENT_URI                              STATE
v01   gs://tsaikevin-1236-90830/model_repository  CREATING


#### Run prediction using `curl`

The "simple" model takes two tensors with shape [1,16] and does a couple of basic arithmetic operation.

In [ ]:
!curl -X POST $ENDPOINT/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$VERSION_NAME:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -d @simple.json

#### Run prediction using Using `requests` library

In [ ]:
import requests
import os
import json

with open('simple.json', 'r') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, MODEL_NAME, VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

### ResNet-50 model (binary data)

#### Create Model

In [38]:
BINARY_MODEL_NAME='resnet50_netdef'

In [39]:
!gcloud ai-platform models create $BINARY_MODEL_NAME --region $REGION --enable-logging

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ml engine model [projects/tsaikevin-1236/models/resnet50_netdef].


#### Create Model Version

In [40]:
BINARY_VERSION_NAME='v01'

#### Command with YAML config file

In [41]:
import yaml

config_binary={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+BINARY_MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+BINARY_MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_binary.yaml', 'w') as file:
    config_binary = yaml.dump(config_binary, file)

In [42]:
!gcloud beta ai-platform versions create $BINARY_VERSION_NAME \
  --model $BINARY_MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_binary.yaml \
  --region=$REGION \
  --async

Using endpoint [https://us-central1-ml.googleapis.com/]


#### To see details of the Model Version just created

In [43]:
!gcloud ai-platform versions describe $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
acceleratorConfig:
  count: '1'
  type: NVIDIA_TESLA_T4
autoScaling:
  minNodes: 1
container:
  args:
  - tritonserver
  - --model-repository=$(AIP_STORAGE_URI)
  image: us-central1-docker.pkg.dev/tsaikevin-1236/caipcustom/tritonserver:20.06-py3
  ports:
  - containerPort: 8000
createTime: '2020-10-26T08:14:00Z'
deploymentUri: gs://tsaikevin-1236-90830/model_repository
etag: kHgiPLE3j_Y=
machineType: n1-standard-4
name: projects/tsaikevin-1236/models/resnet50_netdef/versions/v01
routes:
  health: /v2/models/resnet50_netdef
  predict: /v2/models/resnet50_netdef/infer
state: CREATING


#### To list all Model Versions and their states in this Model

In [44]:
!gcloud ai-platform versions list --model=$BINARY_MODEL_NAME --region=$REGION

Using endpoint [https://us-central1-ml.googleapis.com/]
NAME  DEPLOYMENT_URI                              STATE
v01   gs://tsaikevin-1236-90830/model_repository  CREATING


#### Run prediction using `curl`

Recall the offset value calcuated above.  The binary case has an additional header:

`Inference-Header-Content-Length: [offset]`

In [ ]:
!curl --request POST $ENDPOINT/projects/$PROJECT_ID/models/$BINARY_MODEL_NAME/versions/$BINARY_VERSION_NAME:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat

#### Run prediction using Using `requests` library

In [ ]:
import requests
import os
import json

with open('payload.dat', 'rb') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, BINARY_MODEL_NAME, BINARY_VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Inference-Header-Content-Length': '138',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

## Clean up

In [26]:
!gcloud ai-platform versions delete $VERSION_NAME --model=$MODEL_NAME --region=$REGION --quiet

Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting version [v01]......done.                                              


In [29]:
!gcloud ai-platform models delete $MODEL_NAME --region=$REGION --quiet

Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting model [simple]...done.                                                


In [27]:
!gcloud ai-platform versions delete $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION --quiet

Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting version [v01]......done.                                              


In [28]:
!gcloud ai-platform models delete $BINARY_MODEL_NAME --region us-central1 --region=$REGION --quiet

Using endpoint [https://us-central1-ml.googleapis.com/]
Deleting model [resnet50_netdef]...done.                                       


In [ ]:
!gsutil -m rm -r -f $MODEL_BUCKET

In [8]:
!rm -rf model_repository triton-inference-server *.yaml *.dat *.json